Librerias

In [1]:
import pandas as pd
import xgboost as xgb
from numpy import array
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from urllib.request import urlretrieve

Lectura datos

In [2]:
def ver_contenido(ruta, mode="r", lineas=10):
    with open(ruta) as archivo:
        contenido = archivo.readlines()
        [print(i) for i in contenido[:lineas]]
        
ver_contenido("CSVS STATS/todas_stats.csv")

equipo,victorias_esta_temp,empates_esta_temp,derrotas_esta_temp,porcentaje_victorias_esta_temp,porcentaje_empates_esta_temp,porcentaje_derrotas_esta_temp,victorias_ult10_temp,empates_ult10_temp,derrotas_ult10_temp,porcentaje_victorias_ult10_temp,porcentaje_empates_ult10_temp,porcentaje_derrotas_ult10_temp,Goles Marcados Local_esta_temp,Goles Marcados Visitante_esta_temp,Goles Recibidos Local_esta_temp,Goles Recibidos Visitante_esta_temp,Goles Marcados Local_ult10_temp,Goles Marcados Visitante_ult10_temp,Goles Recibidos Local_ult10_temp,Goles Recibidos Visitante_ult10_temp,Golesxpartido Local_ult10_temp,Golesxpartido Visitante_ult10_temp,Golesxpartido Recibidos Local_ult10_temp,Golesxpartido Recibidos Visitante_ult10_temp,titulos_UCL_ganados,titulos_UCL_ult10_temp

Copenhague,2.0,2.0,2.0,33.3,33.3,33.3,2.0,2.0,2.0,33.3,33.3,33.3,6,2,5,3,6.0,2.0,5.0,3.0,2.0,0.7,1.7,1.0,0,0

Leipzig,4.0,0.0,2.0,66.7,0.0,33.3,22.0,5.0,19.0,47.8,10.9,41.3,6,7,5,5,44.0,37.0,37.0,48.0,1.8,1.7,1.5,2.2,0,0

PSG

XGBoost requiere matrices numéricas para funcionar correctamente, de modo que debemos convertir nuestras columnas de datos de tipo cadena de texto a tipo numérico.

In [4]:
agar = pd.read_csv("CSVS STATS/todas_stats.csv")
print(agar.head())

       equipo  victorias_esta_temp  empates_esta_temp  derrotas_esta_temp  \
0  Copenhague                  2.0                2.0                 2.0   
1     Leipzig                  4.0                0.0                 2.0   
2         PSG                  2.0                2.0                 2.0   
3       Lazio                  3.0                1.0                 2.0   
4         PSV                  2.0                3.0                 1.0   

   porcentaje_victorias_esta_temp  porcentaje_empates_esta_temp  \
0                            33.3                          33.3   
1                            66.7                           0.0   
2                            33.3                          33.3   
3                            50.0                          16.7   
4                            33.3                          50.0   

   porcentaje_derrotas_esta_temp  victorias_ult10_temp  empates_ult10_temp  \
0                           33.3                   2.0  

In [8]:
def str_a_num(df):
    for col in df:
        original = df[col].unique()
        reemplazo = list(range(len(original)))
        mapa = dict(zip(original, reemplazo))
        df[col] = df[col].replace(mapa)
    return(df)

agar = str_a_num(agar)
agar

,equipo,victorias_esta_temp,empates_esta_temp,derrotas_esta_temp,porcentaje_victorias_esta_temp,porcentaje_empates_esta_temp,porcentaje_derrotas_esta_temp,victorias_ult10_temp,empates_ult10_temp,derrotas_ult10_temp,...,Goles Marcados Local_ult10_temp,Goles Marcados Visitante_ult10_temp,Goles Recibidos Local_ult10_temp,Goles Recibidos Visitante_ult10_temp,Golesxpartido Local_ult10_temp,Golesxpartido Visitante_ult10_temp,Golesxpartido Recibidos Local_ult10_temp,Golesxpartido Recibidos Visitante_ult10_temp,titulos_UCL_ganados,titulos_UCL_ult10_temp
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,1,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0,0
3,3,2.0,2.0,0.0,2.0,2.0,0.0,3.0,1.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,0,0
4,4,0.0,3.0,1.0,0.0,3.0,1.0,0.0,3.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,0,0
5,5,2.0,3.0,2.0,2.0,3.0,2.0,4.0,4.0,5.0,...,5.0,5.0,5.0,5.0,4.0,5.0,5.0,5.0,1,0
6,6,1.0,1.0,0.0,1.0,1.0,0.0,4.0,1.0,6.0,...,6.0,6.0,6.0,6.0,5.0,6.0,6.0,5.0,0,0
7,7,2.0,2.0,0.0,2.0,2.0,0.0,5.0,0.0,3.0,...,7.0,7.0,7.0,7.0,6.0,7.0,4.0,6.0,0,0
8,8,3.0,2.0,2.0,3.0,2.0,2.0,6.0,5.0,7.0,...,8.0,8.0,8.0,8.0,7.0,8.0,2.0,0.0,2,1
9,9,2.0,3.0,2.0,2.0,3.0,2.0,7.0,3.0,8.0,...,4.0,9.0,9.0,9.0,4.0,3.0,2.0,7.0,0,0


In [9]:
agar_train, agar_test = train_test_split(agar, test_size=.3, random_state=1999)

In [10]:
agar_train.shape

(11, 27)

In [11]:
agar_test.shape

(5, 27)

In [12]:
agar_train_mat = xgb.DMatrix(agar_train.drop("equipo", 1), label=agar_train["equipo"])
agar_test_mat = xgb.DMatrix(agar_test.drop("equipo", 1), label=agar_test["equipo"])

C:\Users\usuario\AppData\Local\Temp\ipykernel_22000\2540537835.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  agar_train_mat = xgb.DMatrix(agar_train.drop("equipo", 1), label=agar_train["equipo"])
C:\Users\usuario\AppData\Local\Temp\ipykernel_22000\2540537835.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  agar_test_mat = xgb.DMatrix(agar_test.drop("equipo", 1), label=agar_test["equipo"])


In [13]:
agar_train_mat

In [14]:
parametros = {"booster":"gbtree", "max_depth": 2, "eta": 0.3, "objective": "binary:logistic", "nthread":2}
rondas = 10

In [15]:
evaluacion = [(agar_test_mat, "eval"), (agar_train_mat, "train")]

In [16]:
modelo = xgb.train(parametros, agar_test_mat, rondas, evaluacion)

c:\Users\usuario\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:727: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


XGBoostError: [17:39:18] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:144: label must be in [0,1] for logistic regression